In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F # For convolution
import numpy as np
from collections import deque
import multiprocessing
from tqdm.notebook import tqdm # For progress bar (optional, replace with tqdm if not in notebook)
import time
import math

# Optimization 1: Vectorized MRF Update using Convolution (Checkerboard)
# Optimization 2: Batch Generation

def generate_mrf_batched(
    batch_size,
    size=50,
    interaction_strength=1.0,
    external_field=0.0,
    num_iterations=1000, # Note: Iterations here mean full grid updates (2 half-steps)
    device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Генерирует батч бинарных матриц размера size x size с использованием модели Изинга (MRF)
    с использованием векторизованного обновления (шахматная доска) на GPU.

    Args:
        batch_size (int): Количество карт для генерации в батче.
        size (int): Размер матрицы (size x size).
        interaction_strength (float): Сила взаимодействия (J).
        external_field (float): Внешнее поле (h).
        num_iterations (int): Количество полных итераций обновления сетки.
        device (str): Устройство для вычислений ('cuda' или 'cpu').

    Returns:
        torch.Tensor: Батч бинарных матриц (0 и 1) размера (batch_size, size, size) на указанном устройстве.
    """
    # Инициализация батча случайных бинарных матриц (-1 и 1)
    state = torch.randint(0, 2, (batch_size, 1, size, size), dtype=torch.float32, device=device) * 2 - 1

    # Ядро для суммирования соседей по фон Нейману
    kernel = torch.tensor([[0, 1, 0],
                           [1, 0, 1],
                           [0, 1, 0]], dtype=torch.float32, device=device).reshape(1, 1, 3, 3)

    # Маски для шахматного обновления
    mask_white = torch.zeros((size, size), dtype=torch.bool, device=device)
    mask_white[::2, ::2] = 1
    mask_white[1::2, 1::2] = 1
    mask_black = ~mask_white

    # Расширяем маски до размера батча
    mask_white = mask_white.unsqueeze(0).unsqueeze(0) # Shape (1, 1, size, size)
    mask_black = mask_black.unsqueeze(0).unsqueeze(0) # Shape (1, 1, size, size)

    for _ in range(num_iterations):
        # --- Обновление "белых" клеток ---
        # Вычисление суммы соседей для всех клеток одновременно
        neighbor_sum = F.conv2d(state, kernel, padding=1) # padding=1 handles boundaries

        # Локальное поле для всех клеток
        local_field = external_field + interaction_strength * neighbor_sum

        # Вероятность *не* переворота (остаться в текущем состоянии) P(s_i) ~ exp(-E_i)
        # P(spin = s) = sigmoid(2 * s * local_field)
        # Мы используем s = state[i, j]
        prob_stay = torch.sigmoid(2 * state * local_field)

        # Случайные числа для принятия решения
        random_numbers = torch.rand_like(state)

        # Обновляем только "белые" клетки: переворачиваем, если random_number > prob_stay
        flip_condition_white = (random_numbers > prob_stay) & mask_white
        state[flip_condition_white] *= -1

        # --- Обновление "черных" клеток ---
        # Повторное вычисление суммы соседей (т.к. белые могли измениться)
        neighbor_sum = F.conv2d(state, kernel, padding=1)
        local_field = external_field + interaction_strength * neighbor_sum
        prob_stay = torch.sigmoid(2 * state * local_field)
        random_numbers = torch.rand_like(state)

        # Обновляем только "черные" клетки
        flip_condition_black = (random_numbers > prob_stay) & mask_black
        state[flip_condition_black] *= -1

    # Преобразование к бинарным значениям (0 и 1) и удаление канала
    binary_matrix_batch = ((state + 1) / 2).squeeze(1)
    return binary_matrix_batch


# Функция проверки перколяции остается без изменений, т.к. работает на CPU с numpy
def check_percolation(binary_map):
    """
    Проверяет наличие перколяции (связного пути из 1 от левой до правой границы)
    в бинарной матрице.

    Args:
        binary_map (np.array): Бинарная матрица (0 и 1).

    Returns:
        int: 1, если перколяция есть, 0 в противном случае.
    """
    rows, cols = binary_map.shape
    if np.sum(binary_map[:, 0]) == 0: # Оптимизация: если нет 1 в первом столбце, перколяции нет
        return 0

    visited = np.zeros((rows, cols), dtype=bool)
    queue = deque()

    # Начало поиска из первого столбца
    for r in range(rows):
        if binary_map[r, 0] == 1: # and not visited[r, 0] (не нужно, т.к. visited[r,0] будет False)
            queue.append((r, 0))
            visited[r, 0] = True

    while queue:
        current_row, current_col = queue.popleft()

        # Достигли правой границы
        if current_col == cols - 1:
            return 1

        # Исследование соседей (оптимизировано)
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]: # Право, низ, лево, верх
            neighbor_row, neighbor_col = current_row + dr, current_col + dc

            # Проверка границ и условий (оптимизировано)
            if (0 <= neighbor_row < rows and
                0 <= neighbor_col < cols and
                binary_map[neighbor_row, neighbor_col] == 1 and
                not visited[neighbor_row, neighbor_col]):
                visited[neighbor_row, neighbor_col] = True
                queue.append((neighbor_row, neighbor_col))

    return 0

# Обертка для использования с multiprocessing.Pool
def check_percolation_wrapper(args):
    index, binary_map = args
    return index, check_percolation(binary_map)

# Optimization 3: Parallel Percolation Check using multiprocessing
# Optimization 4: Batched GPU -> CPU Transfer
# Optimization 5: Pre-allocation

def create_percolation_dataset_optimized(
    num_samples,
    size=50,
    interaction_strength=1.0,
    external_field=0.0,
    mrf_iterations=100, # Может потребоваться меньше итераций для сходимости с конволюцией
    batch_size=64,      # Размер батча для GPU и параллельной обработки CPU
    num_workers=None,   # Количество CPU процессов (None = cpu_count())
    device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Создает датасет из бинарных карт (X) и их статуса перколяции (y),
    используя оптимизированные методы генерации и проверки.

    Args:
        num_samples (int): Количество образцов для генерации.
        size (int): Размер карт (size x size).
        interaction_strength (float): Сила взаимодействия в MRF.
        external_field (float): Внешнее поле в MRF.
        mrf_iterations (int): Количество итераций для генерации MRF.
        batch_size (int): Размер батча для обработки.
        num_workers (int, optional): Количество процессов для проверки перколяции. Defaults to os.cpu_count().
        device (str): Устройство для генерации MRF ('cuda' или 'cpu').

    Returns:
        tuple: Кортеж из двух numpy.ndarray:
            - X: Массив бинарных карт размера (num_samples, size, size).
            - y: Массив меток перколяции размера (num_samples,).
    """
    if num_workers is None:
        num_workers = multiprocessing.cpu_count()
        print(f"Using {num_workers} workers for percolation check.")

    # Пре-аллокация памяти
    X_data = np.zeros((num_samples, size, size), dtype=np.float32) # Используем float32 для совместимости с PyTorch/np
    y_labels = np.zeros(num_samples, dtype=np.int8) # int8 достаточно для 0/1

    num_batches = math.ceil(num_samples / batch_size)
    generated_count = 0

    # Создаем пул процессов один раз
    with multiprocessing.Pool(processes=num_workers) as pool:
        with tqdm(total=num_samples, desc="Generating dataset") as pbar:
            for i in range(num_batches):
                current_batch_size = min(batch_size, num_samples - generated_count)
                if current_batch_size <= 0:
                    break

                # Шаг 1: Генерация батча карт на GPU
                mrf_maps_batch_tensor = generate_mrf_batched(
                    batch_size=current_batch_size,
                    size=size,
                    interaction_strength=interaction_strength,
                    external_field=external_field,
                    num_iterations=mrf_iterations,
                    device=device
                )

                # Шаг 2: Перенос батча на CPU (один раз для всего батча)
                # Используем .numpy(force=True) с PyTorch 1.12+ или .cpu().numpy()
                try:
                     mrf_maps_batch_cpu = mrf_maps_batch_tensor.cpu().numpy()
                except AttributeError: # older Pytorch?
                     mrf_maps_batch_cpu = mrf_maps_batch_tensor.numpy(force=True)


                # Шаг 3: Параллельная проверка перколяции на CPU
                # Подготовка аргументов для pool.imap_unordered
                tasks = [(generated_count + j, mrf_maps_batch_cpu[j]) for j in range(current_batch_size)]

                # Используем imap_unordered для получения результатов по мере готовности
                for idx, percolates in pool.imap_unordered(check_percolation_wrapper, tasks):
                    X_data[idx] = mrf_maps_batch_cpu[idx - generated_count] # Сохраняем карту
                    y_labels[idx] = percolates                           # Сохраняем метку

                generated_count += current_batch_size
                pbar.update(current_batch_size)

    return X_data, y_labels


# --- Пример использования ---
if __name__ == '__main__':
    # Запускаем через if __name__ == '__main__': для совместимости с multiprocessing
    multiprocessing.freeze_support() # Для Windows

    num_data_points = 1000 # Увеличим для демонстрации скорости
    map_size = 50
    interaction = 0.8  # Пример значения (подберите для вашей задачи)
    external_h = 0.0
    # Для MCMC с конволюцией может потребоваться меньше итераций для "смешивания"
    mrf_iters = 100
    gpu_batch_size = 128 # Настройте в зависимости от VRAM
    cpu_workers = 8    # Настройте в зависимости от количества ядер CPU

    print("Starting Optimized Dataset Generation...")
    start_time = time.time()

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    if device == 'cuda':
         print(f"GPU Name: {torch.cuda.get_device_name(0)}")


    X_data_opt, y_labels_opt = create_percolation_dataset_optimized(
        num_samples=num_data_points,
        size=map_size,
        interaction_strength=interaction,
        external_field=external_h,
        mrf_iterations=mrf_iters,
        batch_size=gpu_batch_size,
        num_workers=cpu_workers,
        device=device
    )

    end_time = time.time()
    print(f"\nOptimized generation finished in {end_time - start_time:.2f} seconds.")

    print(f"Размер набора карт (X): {X_data_opt.shape}")
    print(f"Тип данных X: {X_data_opt.dtype}")
    print(f"Размер меток перколяции (y): {y_labels_opt.shape}")
    print(f"Тип данных y: {y_labels_opt.dtype}")
    # print("Пример меток:", y_labels_opt[:20])
    print(f"Доля перколирующих карт: {np.mean(y_labels_opt):.3f}")

    # --- Сравнение с оригинальным (если нужно) ---
    # print("\nStarting Original Dataset Generation...")
    # start_time_orig = time.time()
    # X_data_orig, y_labels_orig = create_percolation_dataset(
    #     num_samples=num_data_points,
    #     interaction_strength=interaction,
    #     external_field=external_h,
    #     mrf_iterations=2000, # Оригинальное количество итераций
    #     device=device
    # )
    # end_time_orig = time.time()
    # print(f"Original generation finished in {end_time_orig - start_time_orig:.2f} seconds.")
    # print(f"Размер набора карт (X_orig): {X_data_orig.shape}")
    # print(f"Размер меток перколяции (y_orig): {y_labels_orig.shape}")
    # print(f"Доля перколирующих карт (orig): {np.mean(y_labels_orig):.3f}")